In [1]:
import numpy as np
import pandas as pd
import pickle

In [2]:
true_df = pd.read_csv('True.csv', quoting = 2, delimiter=',')
fake_df = pd.read_csv('Fake.csv', quoting = 2, delimiter=',')

In [3]:
fake_df.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true_df.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
true_df['label'] = 1
fake_df['label'] = 0

In [6]:
combined_df = pd.concat([true_df, fake_df])

In [7]:
combined_df.head()

,title,text,subject,date,label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [8]:
combined_df.shape

(44898, 5)

In [9]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44898 entries, 0 to 23480
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   label    44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 2.1+ MB


In [10]:
print(combined_df.index)

Index([    0,     1,     2,     3,     4,     5,     6,     7,     8,     9,
       ...
       23471, 23472, 23473, 23474, 23475, 23476, 23477, 23478, 23479, 23480],
      dtype='int64', length=44898)


In [11]:
combined_df = combined_df.reset_index(drop=True)

In [12]:
len(combined_df)

44898

In [13]:
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, 44898):
    review = combined_df['text'][i]
    if not isinstance(review, str):
        review = ''
    review = re.sub('[^a-zA-Z]', ' ', review)
    review = review.lower()
    review = review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
    review = ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
len(corpus)

44898

In [15]:
print(pd.DataFrame(corpus))

                                                       0
0      washington reuter head conserv republican fact...
1      washington reuter transgend peopl allow first ...
2      washington reuter special counsel investig lin...
3      washington reuter trump campaign advis georg p...
4      seattl washington reuter presid donald trump c...
...                                                  ...
44893  st centuri wire say wire report earlier week u...
44894  st centuri wire say familiar theme whenev disp...
44895  patrick henningsen st centuri wirerememb obama...
44896  st centuri wire say al jazeera america go hist...
44897  st centuri wire say wire predict new year look...

[44898 rows x 1 columns]


In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 40000)
X = cv.fit_transform(corpus).toarray()
y = combined_df.iloc[:, -1].values

In [17]:
from sklearn.preprocessing import LabelEncoder
y = y.reshape(len(y), 1)
le = LabelEncoder()
y = le.fit_transform(y)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = 42)

In [19]:
import tensorflow as tf

In [20]:
ann = tf.keras.models.Sequential()

In [21]:
ann.add(tf.keras.layers.Dense(units = 12, activation = 'relu'))

In [22]:
ann.add(tf.keras.layers.Dense(units = 12, activation = 'relu'))

In [23]:
ann.add(tf.keras.layers.Dense(units = 12, activation = 'relu'))

In [24]:
ann.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

In [25]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [26]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 10)

Epoch 1/10
1123/1123 [==============================] - 6s 5ms/step - loss: 0.0894 - accuracy: 0.9742
Epoch 2/10
1123/1123 [==============================] - 6s 5ms/step - loss: 0.0157 - accuracy: 0.9974
Epoch 3/10
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0048 - accuracy: 0.9991
Epoch 4/10
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0037 - accuracy: 0.9992
Epoch 5/10
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0049 - accuracy: 0.9993
Epoch 6/10
1123/1123 [==============================] - 6s 5ms/step - loss: 0.0039 - accuracy: 0.9992
Epoch 7/10
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0016 - accuracy: 0.9998
Epoch 8/10
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0043 - accuracy: 0.9990
Epoch 9/10
1123/1123 [==============================] - 5s 5ms/step - loss: 0.0017 - accuracy: 0.9997
Epoch 10/10
1123/1123 [==============================] - 5s 5ms/step - loss: 5.560

In [27]:
from sklearn.metrics import accuracy_score
y_pred = ann.predict(X_test)
y_pred = (y_pred > 0.5)
accuracy_score(y_test, y_pred)

281/281 [==============================] - 1s 2ms/step


0.9927616926503341

In [28]:
from tensorflow.keras.models import save_model
save_model(ann, 'my_model.h5')

<ipython-input-28-0bad3de025dc>:2: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  save_model(ann, 'my_model.h5')


In [29]:
with open('countvectorizer.pkl', 'wb') as f:
    pickle.dump(cv, f)